In [1]:
import time, pickle, glob, os
from collections import defaultdict

## Rule mining using ECLAT
Notebook taken from a Data Mining course assignment<br>
This implementation is my own, but has been influenced by the implementation of Michael Mampaey, available at http://adrem.uantwerpen.be/sites/adrem.ua.ac.be/files/code/eclat.py

### First pass:
#### Getting the tidlists for the frequent items 

In [2]:
def ext_get_items(files, transaction_attrs, item_attrs, batched=False):
    """
        Returns tidlists of items in the dataframe. 
        Supports items and transactions with multiple attributes.
        Also supports multiple 'types' of items.
        Arguments:
            * fname: if string, path to data file
                     if list, list of paths to data files
            * transaction_attrs: list of attributes (as strings) that make up the transactions
            * item_attrs: nested list of attributes that make up the item(s). Example: [["ID", "event"], ["cat", "event"]]
            * batched: bool, if True, will save tidlists of each file separately & merge them at the end
                       (Note: significantly slower, only use when reading more than ~3 files)
    """
    
    tidlists = defaultdict(set)
    pname = lambda s : s + '-tidlists'
    
    if type(files) == str:
        files = [files]
    
    for fname in files:
        print(f"Reading file '{fname}'...")
        ftidlists = defaultdict(set) # Write tidlists of this file to separate dict
        with open(fname, 'r') as data:
            # Read header of file, which gives us column names & indices
            columns = data.readline().strip().split(',')
            # Read rest of file line by line, collecting items & transactions
            for line in data:
                line = line.strip().split(',') # list with values on this row
                # Collect all items in this row
                row_items = list()
                for itemtype in item_attrs:
                    itemattrs = tuple([line[columns.index(i_at)] for i_at in itemtype])
                    row_items.append(frozenset([itemattrs]))
                # Find transaction of this row
                transaction = frozenset([line[columns.index(t_at)] for t_at in transaction_attrs])

                # Add transaction to items
                for item in row_items:
                    if batched:
                        ftidlists[item].add(transaction)
                    else:
                        tidlists[item].add(transaction)
        
        # Pickle & store tidlists found in this file
        if batched:
            with open(pname(fname), 'wb') as pfile:
                pickle.dump(ftidlists, pfile)
            
    if batched:
        # Merge all dictionaries into one again
        def merge(new, origin={}):
            for i in new:
                if i in origin:
                    origin[i] = origin[i].union(new[i]) # merge tidlists
                else:
                    origin[i] = new[i]

        print("Merging tidlists...")
        for fname in files:
            with open(pname(fname), 'rb') as pfile:
                ftidlists = pickle.load(pfile)
                # Merge into existing
                merge(ftidlists, tidlists)
            
    # Convert dict to list
    print("Finalizing...")
    return list(tidlists.items())


def merge_tidlists(fnames):
    """
        Helper function. Just the merging part in the ext_get_items() function, for quicker tidlist loading.
    """
    def merge(new, origin={}):
        for i in new:
            if i in origin:
                origin[i] = origin[i].union(new[i]) # merge tidlists
            else:
                origin[i] = new[i]

    tidlists = defaultdict(set)
    print("Merging tidlists...")
    for fname in files:
        with open(pname(fname), 'rb') as pfile:
            ftidlists = pickle.load(pfile)
            # Merge into existing
            merge(ftidlists, tidlists)
    return list(tidlists.items())

#### Keeping only the items above minimum support

In [3]:
def prune_items(items, min_sup):
    """
    Prunes non-frequent items, given a minimum support.
    Arguments:
            * min_sup: int, interpreted as amount of transactions; 
    """
    return list(filter(lambda item : len(item[1]) >= min_sup, items))

In [4]:
datapath = "./data/tags"
pathlist = ["tag-transactions"]#glob.glob(os.path.join(str(datapath), "*.csv"))
pathlist

['tag-transactions']

In [5]:
# Parameters
transaction_attrs = ["v_id"]
item_attrs = [["tag"]]

In [29]:
starttime = time.time()
print("Collecting tidlists...")
all_tidlists = ext_get_items(pathlist, transaction_attrs, item_attrs, True)
print(f"Done. Took {time.time()-starttime:.2f} seconds.")

Reading file 'tag-transactions'...
Merging tidlists...
Finalizing...
Done. Took 0.03 seconds.


In [30]:
MIN_SUP = 7
tidlists = prune_items(all_tidlists, MIN_SUP)
print(f"Found {len(tidlists)} frequent items for min. sup. = {MIN_SUP}")

Found 74 frequent items for min. sup. = 7


In [31]:
# Filtering certain keywords to reduce scale of search
def prune_keywords(tidlists, keywords):
    cleaned = list()
    for i in tidlists:
        item = list(i[0])[0][0]
        if item not in keywords:
            cleaned.append(i)
    return cleaned
blacklist = ["#karen", "#karens", "#vaccinated", "#nissan", "#6inch", "#4x4", "#35s",
             "#6inch35s", "#4x4girls", "#np300", "#navara"]
cleaned_tidlists = prune_keywords(tidlists, set(blacklist))
len(cleaned_tidlists)

74

### Subsequent passes:
#### Recursively mining frequent sets

In [9]:
def mine_sets(min_sup, tidlists=[], depth=0, max_depth=None, frequents={}):
    """
        Recursively mines frequent itemsets.
        Arguments:
            * tidlists: list of pairs (itemset, tidlist)
            * min_sup: int indicating minimum support
        Returns:
            Dict of frequent itemsets and their supports
    """
    if max_depth is not None and depth >= max_depth:
        return frequents
        
    # Intersect 'first' items
    for i in range(len(tidlists)):
        # If first pass, add frequent (single) items
        if depth == 0 and len(tidlists[i][1]) >= min_sup:
            frequents[tidlists[i][0]] = len(tidlists[i][1])
        
        # Create conditional database on this item
        cond_db = list()
        # With all other/next items
        for j in range(i+1, len(tidlists)):
            # Intersect sets
            cand = tidlists[i][0].union(tidlists[j][0])
            cand_tids = tidlists[i][1].intersection(tidlists[j][1])
                        
            # Check support
            sup = len(cand_tids)
            if sup >= min_sup:
                cond_db.append((cand, cand_tids))
                frequents[cand] = sup
                
        # Recursively mine conditional database
        i_cond_sets = mine_sets(min_sup, cond_db, depth+1, max_depth, frequents)
        
    return frequents
                

In [32]:
starttime = time.time()
sets = defaultdict(int)
sets = mine_sets(MIN_SUP, cleaned_tidlists, frequents=sets)
print(f"Found {len(sets)} frequent sets in {time.time()-starttime:.2f} seconds.")

Found 558 frequent sets in 0.01 seconds.


In [11]:
def item_counts(sets):
    # Returns dict of item counts
    counts = defaultdict(int)
    for s in sets:
        items = [x[0] for x in list(s)]
        for i in items:
            counts[i] += 1
    return counts

In [12]:
ics = item_counts(sets)

In [13]:
for i in ics:
    if ics[i] > 20:
        print(i, ics[i])

#thirdposition 86
#based 40
#rightwing 66
#farright 67
#politcs 80
#politicstiktok 80
#reactionary 80
#uk 32
#england 80
#nationalism 80
#traditionalism 50
#czech 63
#rejectmodernity 34
#embracetradition 32
#catholic 32
#history 48
#whitepeoplethings 63
#europeforeuropean 64
#europeanbeauty 51
#whiteculture 60
#europeans 59
#proudtobewhite 48


### Rule mining

In [33]:
"""
    Powerset function, copied from Mark Rushakoff at:
    https://stackoverflow.com/questions/1482308/how-to-get-all-subsets-of-a-set-powerset
"""
from itertools import chain, combinations
def powerset(iterable):
    "powerset([1,2,3]) --> (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)+1))


def find_rules(freq_items, min_conf):
    """
        Finds rules in the given itemsets.
        Arguments:
            * freq_items: Dict with itemsets and their confidence
            * min_conf: Minimum confidence for a rule.
    """
    rules = list()
    for itemset in freq_items:
        if len(itemset) == 1:
            continue # Skip single frequent items
        # Split itemset up into all possible rules
        subsets = list(powerset(itemset))
        # Check rule from every subset to every other subset
        set_conf = freq_items[itemset]
        for ss_X in subsets:
            ss_X = frozenset(ss_X)
            
            if len(ss_X) == len(itemset):
                continue # Skip rules that have no endpoint
            
            X_conf = freq_items[ss_X]
            rule_conf = float(set_conf) / float(X_conf)
            
            if rule_conf >= min_conf:
                ss_Y = itemset - ss_X
                rules.append(((ss_X, ss_Y), rule_conf))
    return rules

In [34]:
starttime = time.time()
min_conf = 0.3
rules = find_rules(sets, min_conf)
print(f"Found {len(rules)} rules in {time.time()-starttime:.2f} seconds.")

Found 4434 rules in 0.03 seconds.


In [35]:
def prune_rules(rules):
    """
        Removes rules that are considered useless.
        This is task-specific and requires actual knowledge about the problem;
        it could be refined further. In this implementation only some base assumptions have been made.
        This implementation also assumes items of the form:
        (product ID, event type) or (category ID, event type)
    """    
    def rule_filter(X, eventX, Y, eventY):
            # returns bool that indicates whether bool violates filter
            return eventX in X and eventY in Y
        
    def filter_terms(terms, X, Y):
        # True if any of the terms appear in either LHS (X) or RHS (Y) of the rule
        return len(set(terms) & (set(X + Y))) > 0
    
    #to_del = set() # List of rules to remove
    cleaned = list() # List of rules to keep
    for r in rules:
        X = [x[0] for x in list(r[0][0])]
        Y = [x[0] for x in list(r[0][1])]
        # Removes rules with 'karen' or 'karens' in them, since they most likely are satirical
        rem_flag = filter_terms(["#karen", "#karens", "#vaccinated"], X, Y)
        if rem_flag:
            #to_del.add(r)
            continue
        
        """
        # Removes rules of type: 'remove -> cart', 'cart -> view', 'purchase -> cart'
        ev_flag = event_filter(X, "remove_from_cart", Y, "cart") \
                or event_filter(X, "cart", Y, "view") \
                or event_filter(X, "purchase", Y, "cart")
        if ev_flag:
            to_del.add(r)
        """
        
        # Heuristically removing 'obvious rules': if conf is too high, it's likely
        # something like "product of category X in cart -> category X in cart"
        conf = r[1]
        max_conf = 0.99
        if conf > max_conf:
            #to_del.add(r)
            continue
        
        # If all filters passed, keep rule
        cleaned.append(r)

    #return list(filter(lambda r : r not in to_del, rules))
    return cleaned
        

In [36]:
starttime = time.time()
final_rules = prune_rules(rules)
print(f"Kept {len(final_rules)} rules. Took {time.time()-starttime:.2f} seconds.")

Kept 2880 rules. Took 0.01 seconds.


In [40]:
final_rules.sort(key=lambda r : r[1], reverse=True)
k = 1000
for r in final_rules[k:k+20]:
    print("Rule:")
    print(f"{set(r[0][0])} => {set(r[0][1])} : {r[1]:.2f}")

Rule:
{('#history',), ('#europeans',), ('#whiteculture',)} => {('#europeanbeauty',)} : 0.70
Rule:
{('#republican',)} => {('#democrat',)} : 0.70
Rule:
{('#europeforeuropean',), ('#europeans',)} => {('#proudtobewhite',), ('#whitepeoplethings',)} : 0.70
Rule:
{('#based',), ('#authright',)} => {('#chad',)} : 0.69
Rule:
{('#based',), ('#authright',)} => {('#nationalism',)} : 0.69
Rule:
{('#based',), ('#czech',)} => {('#embracetradition',)} : 0.69
Rule:
{('#based',), ('#czech',)} => {('#rejectmodernity',), ('#embracetradition',)} : 0.69
Rule:
{('#slav',)} => {('#catholic',), ('#nationalism',)} : 0.69
Rule:
{('#slav',)} => {('#nationalism',), ('#czech',)} : 0.69
Rule:
{('#europeanbeauty',), ('#europeans',), ('#whiteculture',)} => {('#proudtobewhite',), ('#whitepeoplethings',), ('#europeforeuropean',)} : 0.69
Rule:
{('#europeanbeauty',), ('#europeforeuropean',), ('#europeans',), ('#whiteculture',)} => {('#proudtobewhite',), ('#whitepeoplethings',)} : 0.69
Rule:
{('#europeanbeauty',), ('#whitep